In [ ]:
import sys
import os
import datetime as dtm
from subprocess import Popen, PIPE
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from matplotlib.gridspec import GridSpec

## Fetch squeue data and construct a full table ##

In [ ]:
with Popen(['ssh', 'bridges.psc.edu', 'squeue', '--array', '--format', '%all'], stdout=PIPE) as proc:
    recs = proc.stdout.readlines()

time_of_read = dtm.datetime.now()

cols = recs[0].decode().strip().split('|')
print(cols)
recs = recs[1:]

recL = []
for rec in recs:
    recL.append({a : b for a, b in zip(cols, rec.decode().strip().split('|'))})

In [ ]:
def parse_time(tstr):
    if '-' in tstr:
        days, tstr = tstr.split('-')
        days = int(days)
    else:
        days = 0
    words = tstr.split(':')
    tot = 0
    for word in words:
        tot += 60 * tot + int(word)
    tot += 24 * 60 * 60 * days
    return tot

# parse_time('3-00:04:05')

In [ ]:
integer_fields = ['MIN_CPUS', 'MIN_TMP_DISK', 'JOBID', 'PRIORITY', 'CPUS', 'NODES', 'ARRAY_JOB_ID']
time_fields = ['TIME_LIMIT', 'TIME_LEFT', 'TIME']

# Convert strings to appropriate field types in place
for rec in recL:
    for key in integer_fields:
        try:
            rec[key] = int(rec[key])
        except TypeError:
            pass
    for key in time_fields:
        try:
            rec[key] = parse_time(rec[key])
        except ValueError:
            print('time interval conversion failed: %s = %s' % (key, rec[key]))
            print(rec)
            rec[key] = 0
    rec['TIME_SINCE_SUBMIT'] = (time_of_read - dtm.datetime.fromisoformat(rec['SUBMIT_TIME'])).total_seconds()

# print(recL[0])

In [ ]:
fullDF = pd.DataFrame.from_records(recL)
print(fullDF.columns)
#fullDF

In [ ]:
fullDF['JOBS'] = 1
fullDF['CPU_SEC_USED'] = fullDF['CPUS'] * fullDF['TIME']
fullDF['CPU_SEC_REMAIN'] = fullDF['CPUS'] * fullDF['TIME_LEFT']
#fullDF

## Fetch sinfo data and construct a full table ##

In [ ]:
with Popen(['ssh', 'bridges.psc.edu', 'sinfo', '--format', '%all'], stdout=PIPE) as proc:
    recs = proc.stdout.readlines()

time_of_read_sinfo = dtm.datetime.now()

cols = recs[0].decode().strip().split('|')
print(cols)
recs = recs[1:]

recL = []
for rec in recs:
    recL.append({a : b for a, b in zip(cols, rec.decode().strip().split('|'))})

sinfoDF = pd.DataFrame.from_records(recL)

In [ ]:
def split_counts(row):
    fields = row['CPUS(A/I/O/T) '].split('/')
    n_A, n_I, n_O, n_T = [int(fld) for fld in fields]
    return pd.Series({'n_A':n_A, 'n_I':n_I, 'n_O':n_O, 'n_T':n_T})

In [ ]:
df = sinfoDF.apply(split_counts, axis=1)
sinfoDF = pd.concat([sinfoDF, df], axis=1)

## Tables of running jobs by partition and user ##

In [ ]:
runDF = fullDF[fullDF.STATE == 'RUNNING'][['PARTITION', 'USER', 'ACCOUNT', 'CPUS', 'MIN_CPUS', 'NODES',
                                           'JOBS', 'CPU_SEC_USED', 'CPU_SEC_REMAIN']]
for partition, df in runDF.groupby(['PARTITION', 'USER', 'ACCOUNT']).sum().groupby('PARTITION'):
    display(df.sort_values(by=['CPUS', 'JOBS', 'CPU_SEC_REMAIN'], ascending=False))

## Tables of queued jobs by partition and user ##

In [ ]:
waitDF = fullDF[fullDF.STATE != 'RUNNING'][['PARTITION', 'USER', 'ACCOUNT', 'CPUS', 'MIN_CPUS', 'NODES',
                                           'JOBS', 'CPU_SEC_USED', 'CPU_SEC_REMAIN', 'TIME_SINCE_SUBMIT']]
for partition, df in waitDF.groupby(['PARTITION', 'USER', 'ACCOUNT']).sum().groupby('PARTITION'):
    display(df.sort_values(by=['CPUS', 'JOBS', 'CPU_SEC_REMAIN'], ascending=False))

In [ ]:
fullDF.STATE.unique()

## Table of average wait time by user ##

In [ ]:
df = waitDF[waitDF.PARTITION == 'RM-shared'].groupby(['USER', 'ACCOUNT']).sum()
df['MEAN_TIME_SINCE_SUBMIT'] = df['TIME_SINCE_SUBMIT']/df['JOBS']
df.sort_values(['MEAN_TIME_SINCE_SUBMIT'],ascending=False)

## Table of CPU allocations by partition ##

In [ ]:
partitionCPUDF = sinfoDF[['PARTITION ', 'n_A', 'n_I', 'n_O', 'n_T']].groupby(['PARTITION ']).sum()
partitionCPUDF

In [ ]:
def  queue_plot(axes, runDF, waitDF, partition):
    blockL = []
    txtL = []
    baseX = 0.0
    baseY = 0.0
    maxLWd = 0.0
    maxTotHt = 0.0
    df = waitDF[waitDF.PARTITION == partition].groupby(['USER', 'ACCOUNT']).sum()
    df['MEAN_TIME_SINCE_SUBMIT'] = df['TIME_SINCE_SUBMIT']/df['JOBS']
    df.sort_values(['MEAN_TIME_SINCE_SUBMIT'],ascending=False)
    for idx, row in df.reset_index().sort_values(['MEAN_TIME_SINCE_SUBMIT'], ascending=False).iterrows():
        ht = row['CPUS']
        wd = float(row['CPU_SEC_REMAIN'])/(3600. * row['CPUS'])
        rect = Rectangle((baseX, baseY), wd, ht, ec='black')
        blockL.append(axes.add_artist(rect))
        ltxt = plt.Annotation('%s (%s) %.2f hours' % (row['USER'], row['ACCOUNT'],
                                                   row['MEAN_TIME_SINCE_SUBMIT']/3600.0),
                              (baseX + 0.5*wd, baseY + 0.5*ht),
                              va='center', ha='center'
                             )
        txtL.append(axes.add_artist(ltxt))
        baseY += ht
        maxLWd = max(maxLWd, wd)
    maxTotHt = max(maxTotHt, baseY)
    baseX = 0.0
    baseY = 0.0
    maxRWd = 0.0
    df = runDF[runDF.PARTITION == partition].groupby(['USER', 'ACCOUNT']).sum()
    df['MEAN_HOURS_REMAIN'] = df['CPU_SEC_REMAIN']/(3600. * df['CPUS'])
    df.sort_values(['MEAN_HOURS_REMAIN'],ascending=False)
    for idx, row in df.reset_index().sort_values(['CPUS'], ascending=False).iterrows():
        ht = row['CPUS']
        wd = float(row['MEAN_HOURS_REMAIN'])
        rect = Rectangle((baseX - wd, baseY), wd, ht, ec='black', fc='red')
        blockL.append(axes.add_artist(rect))
        ltxt = plt.Annotation('%s (%s)' % (row['USER'], row['ACCOUNT']),
                              (baseX - 0.5*wd, baseY + 0.5*ht),
                              va='center', ha='center'
                             )
        txtL.append(axes.add_artist(ltxt))
        baseY += ht
        maxRWd = max(maxRWd, wd)
    maxTotHt = max(maxTotHt, baseY)
    axes.set_xlim(-maxRWd, maxLWd)
    axes.set_ylim(0.0, maxTotHt)
    axes.set_xlabel('Hours per CPU')
    axes.set_title('%s\n<- Running | Waiting ->' % partition)
    return maxTotHt

#fix, axes = plt.subplots(1)
#queue_plot(axes, runDF, waitDF, 'GPU-shared')
#plt.show()

    

In [ ]:
def cpu_plot(axes, partitionCPUDF, partition):
    axes0.set_xlim(0.0, 1.0)
    baseY = 0.0
    df = partitionCPUDF.reset_index()
    for idx, row in df.iterrows():
        this_pt = row['PARTITION '].strip()
        if this_pt == partition:
            ht = row['n_A']
            rect = Rectangle((0.0, baseY), 1.0, ht, ec='black', fc='cyan')
            axes.add_artist(rect)
            ltxt = plt.Annotation('alloc', (0.5, baseY + 0.5*ht), va='center', ha='center')
            axes.add_artist(ltxt)
            baseY += ht
    
            ht = row['n_I']
            rect = Rectangle((0.0, baseY), 1.0, ht, ec='black', fc='cyan', hatch='/')
            axes.add_artist(rect)
            ltxt = plt.Annotation('idle', (0.5, baseY + 0.5*ht), va='center', ha='center')
            axes.add_artist(ltxt)
            baseY += ht
    
            ht = row['n_O']
            rect = Rectangle((0.0, baseY), 1.0, ht, ec='black', fc='cyan', hatch='+')
            axes.add_artist(rect)
            ltxt = plt.Annotation('other', (0.5, baseY + 0.5*ht), va='center', ha='center')
            axes.add_artist(ltxt)
            baseY += ht
    return baseY
    


In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
%matplotlib inline
#%matplotlib qt
plt.rcParams["figure.figsize"] = [16, 10]

for partition in fullDF.PARTITION.unique():
    fig = plt.figure()
    gs = GridSpec(1, 2, width_ratios=[1, 10])
    gs.update(wspace=0.0)
    axes0 = fig.add_subplot(gs[0])
    axes1 = fig.add_subplot(gs[1], sharey=axes0)
    plt.setp(axes1.get_yticklabels(), visible=False)
    axes0.set_ylabel('CPUs')
    plt.setp(axes0.get_xticklabels(), visible=False)
    axes0.get_xaxis().set_ticks([])
    ht1 = queue_plot(axes1, runDF, waitDF, partition)
    ht2 = cpu_plot(axes0, partitionCPUDF, partition)
    axes1.set_ylim(0.0, max(ht1, ht2))
    plt.show()